### Spammer
Você recebeu uma lista de emails com o nome dos usuários, qual produto eles compraram e seus emails.

Você deverá mandar um email para cada um deles, dando boas vindas. Para que você aumente as chances de abertura do seu email, é interessante que você o personalize. Assim, coloque no título do email o nome do aluno e no corpo, o nome completo.

Neste exemplo, usaremos diretamente a biblioteca do Gmail. Mas [existem outras maneira de fazer](https://nitratine.net/blog/post/how-to-send-an-email-with-python/), talvez até mais simples (usando o protocolo SMTP). Use esta [ajuda](https://pastebin.com/aCQaVavQ) para escrever o seu código

In [1]:
# Insira aqui o seu algoritmo/lógica



In [1]:
import openpyxl
import os
import pandas as pd
import sys

# busca as informações o excel
arquivo = 'listaDeClientes.xlsx'
diretorio = os.path.abspath(os.curdir)
df = pd.read_excel(arquivo)
df.head()

,Nome do Cliente,Nome do Produto,Email
0,Adriano Almeida,Produto 1,email@gg.com
1,Gustavo Gil,Produto 1,email@gg.com
2,Luiza Hilário,Produto 1,email@gg.com
3,Théo Brito,Produto 1,email@gg.com
4,Eloá Coutinho,Produto 1,email@gg.com


In [2]:
# definição dos textos das mensagens
texto_mensagem = """
Olá {},\n

Seja bem vindo à nossa empresa, estamos muito felizes de você estar aqui e fazendo o {}!\n
Precisando, só chamar!\n

Um Abs, \n
Equipe
"""

texto_titulo = """Que felicidade, {}"""

In [3]:
# imports extraídos dos links
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import mimetypes
import os

from apiclient import errors

In [4]:
def criarServicoGmail():    
    """
    Funcao que cria o servico de GMAIL. O servico é a base para acessar o GMAIL
    """
    # IMPORTANTE: Apague o token.pickle se alterar este aqui
    SCOPES = ['https://mail.google.com/']

    creds = None
    # O arquivo token.pickle é criado depois de uma autorização feita
    if os.path.exists('token.pickle'):
        # lê o arquivo e carrega as credenciais já feitas antes
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # verifica se existia um arquivo com as credenciais ou se elas ainda são válidas
    if not creds or not creds.valid:
        # se a credencial expirou (aquele flag de lembre de mim por 30 dias), pede novamente
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # faz o fluxo de autenticacao padrao
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server()
            
        # agora que estamos autenticados, salve para um uso futuro
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # com as credentciais em mãos, crie o servico para acesso ao GMAIL
    service = build('gmail', 'v1', credentials=creds)
    return service


def CreateMessageWithAttachment(sender, to, subject, message_text, file_dir,
                                filename):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
    file_dir: The directory containing the file to be attached.
    filename: The name of the file to be attached.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEMultipart()
    message['To'] = to
    message['From'] = sender
    message['Subject'] = subject

    msg = MIMEText(message_text)
    message.attach(msg)

    path = os.path.join(file_dir, filename)
    content_type, encoding = mimetypes.guess_type(path)

    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'
        main_type, sub_type = content_type.split('/', 1)
    if main_type == 'text':
        fp = open(path, 'rb')
        msg = MIMEText(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'image':
        fp = open(path, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(path, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    else:
        fp = open(path, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()

    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

    return {'raw': base64.urlsafe_b64encode(message.as_string())}

def CreateMessage(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['To'] = to
    message['From'] = sender
    message['Subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('UTF-8')).decode('ascii')}

def SendMessage(service, user_id, message):
    """
    Envia a mensagem

    Args:
    service: servico do GMAIL criado com a funcao CriarServicoGmail
    user_id: email do usuário. 'me' pode ser usado para o seu proprio email
    message: mensagem a ser enviada. Criada com as funcoes CreateMessageWithAttachment ou
    CreateMessage

    Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print('Message Id: %s' % message['id'])
        return message
    except(errors.HttpError, error):
        print('An error occurred: %s' % error)

In [6]:
# criar o servico de Gmail a ser usado
servico = criarServicoGmail()
retorno_msg = None

# para cada uma das linhas do email
for linha in df.iterrows():
    # pega a informação
    data = linha[1]
    
    # cria a mensagem
    nome = data['Nome do Cliente']
    produto = data['Nome do Produto']
    titulo = texto_titulo.format(nome.split()[0])
    mensagem = texto_mensagem.format(nome, produto)
    msg = CreateMessage('adrianowalmeida@gmail.com', data['Email'], titulo, mensagem)
    print("Mandando mensagem para {}".format(data['Email']))
    
    # envia a mensagem
    retorno_msg = SendMessage(service=servico, user_id='me', message=msg)
    break

Mandando mensagem para email@gg.com
Message Id: 169ed30a0bd398d7
